In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.ticker as ticker   


# setup spark
conf = SparkConf().setAppName('WikiGraph')
sc = SparkContext(conf=conf)
#from P5_bfs import *

import pandas as pd

In [2]:
# paths to the big data files
datapath = '../../../../../courses/CS205_Computing_Foundations/data/'
titlespath = datapath + 'titles-sorted.txt'
linkspath = datapath + 'links-simple-sorted.txt'

In [3]:
rddTitles = sc.textFile(titlespath)
rddGraph = sc.textFile(linkspath)

# prepare Graph into adjacency list structure
rddGraph = rddGraph.map(lambda x: x.replace(':', '').split(' ')).map(lambda x: (int(x[0]), [int(y) for y in x[1:]]))

In [4]:
rddTitles.take(5)


[u'!', u'!!', u'!!!', u'!!!!', u'!!!Fuck_You!!!']

In [5]:
rddTitles = rddTitles.zipWithIndex().map(lambda x: (x[0], x[1] + 1)).cache()

In [6]:
rddTitles.take(5)

[(u'!', 1), (u'!!', 2), (u'!!!', 3), (u'!!!!', 4), (u'!!!Fuck_You!!!', 5)]

In [7]:
# lookup a title (its index)
keyString = 'Kevin_Bacon'
v0 = rddTitles.filter(lambda x: x[0] == keyString).collect()[0][1]

keyString = 'Harvard_University'
vT = rddTitles.filter(lambda x: x[0] == keyString).collect()[0][1] 

In [8]:
v0

2729536

In [9]:
vT

2152782

In [10]:
# compute shortest path!
from P5_sssp import *

In [11]:
num_visited_nodes, rddPaths = sparkSSSP(sc, rddGraph, v0, vT, 20)

In [12]:
resultPaths = rddPaths.collect()

In [13]:
resultPaths

[[2729536, 3229511, 2152782],
 [2729536, 4625677, 2152782],
 [2729536, 2578703, 2152782],
 [2729536, 5114592, 2152782],
 [2729536, 1124925, 2152782]]

In [14]:
rddLookup = sc.parallelize(resultPaths)

In [15]:
rddLookup.take(5)

[[2729536, 3229511, 2152782],
 [2729536, 4625677, 2152782],
 [2729536, 2578703, 2152782],
 [2729536, 5114592, 2152782],
 [2729536, 1124925, 2152782]]

In [23]:
#rddLookup.zipWithIndex().flatMap(lambda x: [(y, x[1]+1) for y in x[0]]).join(rddTitles.map(lambda x: (x[1], x[0]))).map(lambda x: (x[1][0], x[1][1])).groupByKey().map(lambda x: list(x[1])).collect()

[[u'Kevin_Bacon', u'Harvard_University', u'Marisa_Silver'],
 [u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'],
 [u'Kevin_Bacon', u'Harvard_University', u'John_Lithgow'],
 [u'Time_(magazine)', u'Kevin_Bacon', u'Harvard_University'],
 [u'Kevin_Bacon', u'College_Bowl', u'Harvard_University']]

In [41]:
rddLookup.zipWithIndex().flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)])rddLookup.zipWithIndex().flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)]).collect()

[(2729536, (1, 1)),
 (3229511, (1, 2)),
 (2152782, (1, 3)),
 (2729536, (2, 1)),
 (4625677, (2, 2)),
 (2152782, (2, 3)),
 (2729536, (3, 1)),
 (2578703, (3, 2)),
 (2152782, (3, 3)),
 (2729536, (4, 1)),
 (5114592, (4, 2)),
 (2152782, (4, 3)),
 (2729536, (5, 1)),
 (1124925, (5, 2)),
 (2152782, (5, 3))]

In [45]:
# for better performance on join, 
rddLookup.cache()
nodesOfInterest = rddLookup.flatMap(lambda x: x).distinct().collect()
selectedTitles = rddTitles.filter(lambda x: x[1] in nodesOfInterest).map(lambda x: (x[1], x[0]))

In [56]:
rddLookup.zipWithIndex() \
         .flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)]) \
         .join(selectedTitles) \
         .map(lambda x: (x[1][0][0], (x[1][0][1], x[1][1]))) \
         .groupByKey().map(lambda x: zip(*sorted(list(x[1])))[1]).collect()
         

[(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [58]:
test = [(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [59]:
test

[(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [69]:
startTitle  = 'Kevin_Bacon'
targetTitle = 'Harvard_University'

with open('output.txt', 'wb') as f:
	f.write('Shortest paths from %s to %s:\n\n' % (startTitle, targetTitle))
	for path in test:
            path = list(path) 
            f.write(path[0])
            for node in path[1:]:
                f.write(' -> ' + node)
            f.write('\n')